## Import needed modules and packages


In [1]:
#import the geomapi package
import os
import ifcopenshell.geom as geom
import math
from scipy.spatial.transform import Rotation as R
from PIL import Image, ImageDraw
import shutil
from rdflib import Graph, URIRef, RDF
import os.path
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import copy
import pye57 
import ifcopenshell
import multiprocessing as mp
import time
from pathlib import Path
from typing import List
import matplotlib.pyplot as plt
import csv
import xlsxwriter

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
from geomapi.tools import validationtools as vt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## Create a SessionNode

In [4]:
path= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"tests",)

projectPath = r"C:\Data\Projects-MEETHET\012203-34-BIM-Colruyt Schoten" #os.path.join(path,'Samples12')

sn = SessionNode(subject=ut.get_folder(projectPath),projectPath= projectPath )
# print(sn.subject)
# print(sn.get_name())
# print(sn.projectPath)

an = Node(subject=sn.subject, projectPath= projectPath)

## Set needed global parameters

In [5]:
#INPUT parameters
sn.ifcPath = None
sn.pcdPath = None

sn.tempPath = None
sn.outputPath = None

# saveCSV = True
# saveExcel = True
# saveColloredBIM = True
# saveColloredPcd = True
# #PARAMETER ADDED
saveReports = True
# saveResultPcd = True

sn.resolution = 0.05

#AVANCED
sn.keys = { 'Wall':["IfcWall", "IfcWindow","IfcDoor"],
         'Column':['IfcColumn'],
         'Beam':["IfcBeam"],
         'Slab':["IfcSlab",'IfcCovering'],
         'Roof':["IfcRoof"]}


an.t30 = 0.015 #LOA30 Treshold
an.t20 = 0.05 #LOA20 Treshold
an.t10 = 0.1 #LOA10 Treshold
an.t00 =1 #Total distance treshold
an.abs = True # use absolute values for percentages

an.Td = 0.1 #distance trechold for filtering
an.Sr=0.1 #Seach radius treshold for normal matching
an.Tdot = 0.8 #treshhold for normal matching 

an.p10 = 0.68 #cumulative percentage treshold LOA10
an.p20 = 0.68 #cumulative percentage treshold LOA20
an.p30 = 0.68 #cumulative percentage treshold LOA30

Create temporary directory to save working files

In [6]:
if not sn.tempPath:
    sn.projectProcessingPath = os.path.join(sn.projectPath, "PROCESSING")
else:
    sn.projectProcessingPath = sn.tempPath

if not os.path.isdir(sn.projectProcessingPath): #check if the folder exists
    os.mkdir(sn.projectProcessingPath)

Create Directory to save results

In [7]:
if not sn.outputPath:
    sn.projectResultsPath = os.path.join(sn.projectPath, "RESULTS")
else:
    sn.projectResultsPath = sn.outputPath

if not os.path.isdir(sn.projectResultsPath): #check if the folder exists
    os.mkdir(sn.projectResultsPath)

resultsName = "session_" + time.strftime("%Y%m%d-%H%M%S")
sn.resultsPath = os.path.join(sn.projectResultsPath, resultsName)
if not os.path.exists(sn.resultsPath):
    os.makedirs(sn.resultsPath)

Get the IFC file

In [8]:
if not sn.ifcPath:
    sn.ifcPath = os.path.join(sn.projectPath, "MODELS") #Construct the data Modelsfolder path
    if os.path.isdir(sn.ifcPath): #Check if the folder exists
        ifc2x3Path = os.path.join(sn.ifcPath, "IFC2x3") #Construct the IFC2x3 folder path
        if os.path.isdir(ifc2x3Path): #check if ther is an IFC2x3 folder present in the project
            content_ifc_dir = os.listdir(ifc2x3Path) #get the content of the folder
            if len(content_ifc_dir) > 0: #check if the folder is not empty
                #Loop over all IFC files in the current directory
                ifcFiles = []
                for ifcfile in content_ifc_dir: #for every ifc model in the folder
                    
                    if ifcfile.endswith(".ifc"):
                        ifcFiles.append(ifcfile)
                    else:
                        print("ERROR: Select an IFC file")
                if len(ifcFiles) > 1:
                    print("ERROR: Specify one IFC file to be processed")
                else:
                    sn.ifcPath = os.path.join(ifc2x3Path, ifcFiles[0])
                    print("Using IFC file: %s" % sn.ifcPath)
            else:
                print("ERROR: Selected Directory is empty %s" % ifc2x3Path)
        else:
            print("ERROR: No such directory %s" % ifc2x3Path)
    else:
        print("ERROR: No such directory %s" % sn.ifcPath)
else:
    if sn.ifcPath.endswith(".ifc"):
        print("Using IFC file: %s" % sn.ifcPath)
    else:
        print("ERROR: Select an IFC file")

ERROR: Select an IFC file
Using IFC file: C:\Data\Projects-MEETHET\012203-34-BIM-Colruyt Schoten\MODELS\IFC2x3\012203-34-BIM-Colruyt_3D model_3.ifc


Get the input pointcloud files

In [9]:
if sn.pcdPath:
    if os.path.isdir(sn.pcdPath):
        dir = sn.pcdPath
        print("All pointclouds in %s will be processed" %dir)
        pcds = os.listdir(dir)
        if len(pcds) > 0:
            sn.pcdPath = []
            for pcd in pcds:
                sn.pcdPath.append(os.path.join(dir,pcd))
    elif os.path.isfile(sn.pcdPath):
        if sn.pcdPath.endswith(".pcd") or sn.pcdPath.endswith(".pts"):
            print("Pointcloud %s will be processed" % sn.pcdPath)
            pcdPath = [sn.pcdPath]
        else:
            print("ERROR: only PCD and PTS format are currently supported")
    else:
        print("ERROR: Please provide an existing path")
    
else:
    dataPath = os.path.join(sn.projectPath, "DATA")
    if os.path.exists(dataPath):
        sn.pcdPath = os.path.join(dataPath, "PCD")
        if os.path.exists(sn.pcdPath):
            dir = sn.pcdPath
            print("All pointclouds in %s will be processed" %dir)
            pcds = os.listdir(dir)
            if len(pcds) > 0:
                sn.pcdPath = []
                for pcd in pcds:
                    if pcd.endswith(".pcd"):
                        sn.pcdPath.append(os.path.join(dir,pcd))
                if not len(sn.pcdPath) > 0:
                    print("ERROR: No PCD Files found")
                    ptsPath = os.path.join(dataPath, "PTS")
                    if os.path.exists(ptsPath):
                        dir = ptsPath
                        print("All pointclouds in %s will be processed" %dir)
                        pcds = os.listdir(dir)
                        if len(pcds) > 0:
                            sn.pcdPath = []
                            for pcd in pcds:
                                if pcd.endswith(".pts"):
                                    sn.pcdPath.append(os.path.join(dir,pcd))
                            if not len(sn.pcdPath) > 0:
                                print("ERROR: No PTS Files found")
        else:
            ptsPath = os.path.join(dataPath, "PTS")
            if os.path.exists(ptsPath):
                dir = ptsPath
                print("All pointclouds in %s will be processed" %dir)
                pcds = os.listdir(dir)
                if len(pcds) > 0:
                    pcdPath = []
                    for pcd in pcds:
                        if pcd.endswith(".pts"):
                            pcdPath.append(os.path.join(dir,pcd))
                    if not len(pcdPath) > 0:
                        print("ERROR: No PTS Files found")
    else:
        print("ERROR: Provide a pointcloud")
        
print(sn.pcdPath)

All pointclouds in C:\Data\Projects-MEETHET\012203-34-BIM-Colruyt Schoten\DATA\PCD will be processed
['C:\\Data\\Projects-MEETHET\\012203-34-BIM-Colruyt Schoten\\DATA\\PCD\\interieur1.pcd', 'C:\\Data\\Projects-MEETHET\\012203-34-BIM-Colruyt Schoten\\DATA\\PCD\\interieur2.pcd', 'C:\\Data\\Projects-MEETHET\\012203-34-BIM-Colruyt Schoten\\DATA\\PCD\\interieur3.pcd', 'C:\\Data\\Projects-MEETHET\\012203-34-BIM-Colruyt Schoten\\DATA\\PCD\\interieur4.pcd', 'C:\\Data\\Projects-MEETHET\\012203-34-BIM-Colruyt Schoten\\DATA\\PCD\\interieur5.pcd']


Additional settings to export the results to excel or csv

In [10]:
# #CSV
csvFilename = "LOA_Report.csv"
csvPath = os.path.join(sn.resultsPath, csvFilename)
header = ['Name','Id', 'Class','LOA', 'LOA10', 'LOA20', 'LOA30']
csvFile = open(csvPath, 'w')
sn.csvWriter = csv.writer(csvFile)
sn.csvWriter.writerow(header)

#Excel
xlsxFilename = "LOA_Report.xlsx"
xlsxPath = os.path.join(sn.resultsPath, xlsxFilename)
sn.workbook = xlsxwriter.Workbook(xlsxPath)
sn.worksheet = sn.workbook.add_worksheet()
sn.worksheet.write(0,0,'Name')
sn.worksheet.write(0,1,'Id')
sn.worksheet.write(0,2,'Class')
sn.worksheet.write(0,3,'LOA')
sn.worksheet.write(0,4, 'LOA10')
sn.worksheet.write(0,5, 'LOA20')
sn.worksheet.write(0,6, 'LOA30')
sn.xlsxRow = 1

## Create some BIMNodes

In [11]:
sn.bimNodes=tl.ifc_to_nodes_multiprocessing(sn.ifcPath)

2. divide and save them per storey

In [12]:
sn.StoreyList=[]
#Open the IFC file to acces the IFC relations
ifc = ifcopenshell.open(sn.ifcPath) 
#Finf all the storeys in a building
for level, ifc_storey in enumerate(ifc.by_type("IfcBuildingStorey")):
    myIds=[]
    
    #find the ids of the elements of that storey
    for reference in ifc.get_inverse(ifc_storey):
        if reference.is_a("IfcRelContainedInSpatialStructure"):
            for product in reference.RelatedElements:
                    if product.is_a("IfcProduct"):  
                        myIds.append(product.GlobalId)

    #select all elements of that storey from bimNodes
    sn.StoreyList.append([n for n in sn.bimNodes if n.globalId in myIds])
    # [n.save_resource(projectProcessingPath) for n in myStoreyList[level] if n.resource is not None]

## Assign keys to bimNodes
Link the label defined in key to the IFCclass

In [13]:
for n in sn.bimNodes:
    found = False
    for key in sn.keys.keys():
        if n.className in sn.keys[key]:
            n.key = key
            found = True
    if not found:
        n.key = 'Clutter'

In [14]:
wallNodes=[n for n in sn.bimNodes if 'Wall' in n.className]
print(len(wallNodes))
columnNodes=[n for n in sn.bimNodes if 'Column' in n.className]
print(len(columnNodes))
beamNodes=[n for n in sn.bimNodes if 'Beam' in n.className]
print(len(beamNodes))
slabNodes=[n for n in sn.bimNodes if 'Slab' in n.className]
print(len(slabNodes))
roofNodes=[n for n in sn.bimNodes if 'Roof' in n.className]
print(len(roofNodes))
clutterNodes=[n for n in sn.bimNodes if 'Clutter' in n.className]
print(len(clutterNodes))

400
57
44
71
586
0


In [15]:
for n in sn.bimNodes:
    n.name = n.key + "_" + n.globalId
    n.path = os.path.join(sn.projectProcessingPath, n.get_name() + ".ply")
    #BUG: the GlobalID as python is Case sensitive but file names ar NOT so when a globalId only differs from another by a capital letter the file was overwriten. Therefore when the file already exists a timesamp is added. This makes the functionalitie that it only saves when theh file doesnt exist unusabel because when the file already exists he will just add a timestamp
    if os.path.exists(n.path):
        n.name = n.key + "_" + n.globalId + time.strftime("%Y%m%d%H%M%S")
    
    if not n.save_resource(sn.projectProcessingPath):
        sn.bimNodes.remove(sn.bimNodes.index(n))

In [16]:
print(len(sn.bimNodes)) 

1484


now that the preprocessing is done, we serialize each nodes' graph

In [17]:
graphPath=os.path.join(projectPath,'meshGraph.ttl')
tl.nodes_to_graph(nodelist=sn.bimNodes,graphPath=graphPath,save=True)

<Graph identifier=Nf3eeb0602f43456cb781322be9ee09d2 (<class 'rdflib.graph.Graph'>)>

In [18]:
print(sn.bimNodes[0].graph.serialize())

@prefix e57: <http://libe57.org#> .
@prefix ifc: <http://ifcowl.openbimstandards.org/IFC2X3_Final#> .
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#> .
@prefix v4d: <https://w3id.org/v4d/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///Surface_2380054_02lVx43qr9L9TaN1LOsKYs> a v4d:BIMNode ;
    ifc:className "IfcSite" ;
    ifc:globalId "02lVx43qr9L9TaN1LOsKYs" ;
    ifc:ifcPath "MODELS\\IFC2x3\\012203-34-BIM-Colruyt_3D model_3.ifc" ;
    e57:cartesianBounds """[ 64.32785024 175.92689883  68.78402476 275.06421504  -1.52915957
   0.64864638]""" ;
    e57:cartesianTransform """[[1.00000000e+00 0.00000000e+00 0.00000000e+00 1.36889052e+02]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 1.61127316e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 1.54342748e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]""" ;
    e57:pointCount 6533 ;
    v4d:faceCount 10642 ;
    v4d:key

if you want to delete all these resources, call del on each node

In [19]:
# for n in bimNodes:
#     del n.resource

woops, now i have to reload all the data

In [20]:
# [n.get_resource() for n in bimNodes]

now if you want to reload all these nodes from graph

In [21]:
# graphPath=os.path.join(projectProcessingPath,'meshGraph.ttl')
# bimNodes=tl.graph_path_to_nodes(graphPath,getResource=True)

In [22]:
print(sn.bimNodes[0].subject)
print(sn.bimNodes[0].resource)

file:///Surface_2380054_02lVx43qr9L9TaN1LOsKYs
TriangleMesh with 6533 points and 10642 triangles.


## Create PCDNodes from MeshNodes

In [23]:
sn.meshpcdNodes=[]
referencepcd = o3d.geometry.PointCloud()
referenceNode = PointCloudNode()
referenceNode.keys = []
referenceNode.objects = 0
for n in sn.bimNodes:
    if not n.resource is None:
        number_of_points=int(n.resource.get_surface_area()/(sn.resolution*sn.resolution))
        myPCD=n.resource.sample_points_uniformly(number_of_points=number_of_points)
        if not myPCD is None:
            myPCDNode=PointCloudNode(resource=myPCD,
                                    name= n.get_name().split("-")[0] + "-MESHPCD",
                                    subject=n.get_name().split("-")[0] + "-MESHPCD",
                                    isDerivedFromGeometry=n.subject,
                                    processed = False)
            myPCDNode.path = os.path.join(sn.projectProcessingPath, myPCDNode.get_name() + ".pcd")
            myPCDNode.save_resource(sn.projectProcessingPath)
            referencepcd.__iadd__(myPCDNode.resource)
            n.object = 1
            ids = len(myPCDNode.resource.points)
            i=0
            while i < ids:
                referenceNode.keys.append(n.object)
                i += 1 


            n.object = referenceNode.objects
            referenceNode.objects += 1

            sn.meshpcdNodes.append(myPCDNode)

referenceNode.resource = referencepcd
print(referencepcd)
print(len(referenceNode.keys))

PointCloud with 22731086 points.
22731086


now that the preprocessing is done, we serialize each nodes' graph

In [24]:
print(len(sn.bimNodes)) #247 mesh files
print(len(sn.meshpcdNodes)) #247 meshpcd files
print(referenceNode.objects)

1484
1484
1484


In [25]:
# graphPath=os.path.join(projectProcessingPath,'meshpcdGraph.ttl')
# tl.nodes_to_graph(nodelist=pcdNodeList,graphPath=graphPath,save=True)

## Create the needed images for the report

1. Function to get the boundingbox of a list of geometries

In [26]:
# def get_boundingbox_of_list_of_geometries(geometries:List[Node]) -> np.array:
#     """Determines the global boundingbox of a group of Node containing geometries.

#     Args:
#         geometries (List[Nodes]):  list of Nodes containing a resource of which the boundingbox must be determined"
            
#     Returns:
#         np.array[3x1]
#     """
#     pcd = o3d.geometry.PointCloud()
#     for n in geometries:
#         n.get_resource()
#         if n.resource is not None:
#             pcd.__iadd__(o3d.geometry.PointCloud(gmu.get_oriented_bounds(gmu.get_cartesian_bounds(n.resource))))
#     cartesianBounds = gmu.get_cartesian_bounds(pcd.get_oriented_bounding_box())
#     return cartesianBounds

2. Determine a good camera stand point for an overview image

In [27]:
# def optimize_camera_for_overview(geometries:List[Node]):
#     """Determines the camera position to create an overview image of a group of Nodes containing geometries.

#     Args:
#         geometries (List[Nodes]):  list of Nodes containing a resource "
            
#     Returns:
#         c center of the objects np.array[1x3]
#         c_i the camera position np.array[1x3]
#         up defines which direction is up [0,0,1] for z
#     """

#     cartesianBounds = get_boundingbox_of_list_of_geometries(geometries=geometries)
#     bbbox = gmu.get_oriented_bounding_box(cartesianBounds)

#     fov = np.pi / 3 #60 #degrees
            
#     #determine extrinsic camera parameters
#     extrinsic = np.empty((1,3), dtype=float)
    
#     c = bbbox.get_center()
#     u = bbbox.extent[0]
#     d_w = math.cos(fov/2)*u

#     #determine c_i
#     rotation_matrix = bbbox.R
#     pcd2 = o3d.geometry.PointCloud()
#     array = np.array([[c[0],c[1],c[2]+d_w]])
#     pcd2.points = o3d.utility.Vector3dVector(array)
#     c_i = np.asarray(pcd2.points[0])

#     up = [0, 0, 1]  # camera orientation

    
#     return c,c_i,up

In [ ]:
print(len(sn.StoreyList))
for storey in sn.StoreyList:
    floor = []
    print(len(storey))
    for object in storey:
        floor.append(object.resource)
    o3d.visualization.draw_geometries(floor)

In [28]:
if saveReports:
    #Create images directory to save results    
    sn.imageResultDirectory = os.path.join(sn.resultsPath, "IMAGES")
    if not os.path.isdir(sn.imageResultDirectory):
        os.mkdir(sn.imageResultDirectory)

    for storey in sn.StoreyList:

        c,c_i,up = val.optimize_camera_for_overview(storey)
        
        #generate scene
        width = 640
        height = 480
        
        for element in storey:
            element.overviewImagePath = None
            if element.resource is not None:
                render = o3d.visualization.rendering.OffscreenRenderer(width,height)
        
                render.scene.camera.look_at(c, c_i, up)
                #Determine path to save the created image
                overviewImageResultName = element.name + "_OVERVIEW"
                overviewImageResultFileName = overviewImageResultName + ".png"
                overviewImageResultPath = os.path.join(sn.imageResultDirectory, overviewImageResultFileName)
                if os.path.exists(overviewImageResultPath):
                    element.name = element.key + "_" + element.globalId + time.strftime("%Y%m%d%H%M%S")
                
                element.resource.paint_uniform_color([1,0,0])
            
            
                material1 = o3d.visualization.rendering.MaterialRecord()
                material1.base_color = [1.0, 0.0, 0.0, 1.0]  # RGBA
                material1.shader = "defaultUnlit"

                render.scene.add_geometry(element.name,element.resource,material1)

                for otherElement in storey:
                    material2 = o3d.visualization.rendering.MaterialRecord()
                    material2.base_color = [0.5, 0.5, 0.5, 0.1]  # RGBA
                    material2.shader = "defaultUnlit"

                    if otherElement is not element and otherElement.resource is not None:
                        render.scene.add_geometry(otherElement.name,otherElement.resource,material2)

                #Take the image
                img = render.render_to_image()
                
                o3d.io.write_image(overviewImageResultPath, img)
                element.overviewImagePath = overviewImageResultPath
                print(element.name)
    

Clutter_2OIUz9zOHD_BYVYmX3KLcB20220912151918
Clutter_1ft2BeOOT9IAqaQNHLnYit20220912151918
Wall_0eCd_c4_jF3fWwzrAmFAGk20220912151918
Clutter_0bOlnkW59DjubTltZzGQ$k20220912151918
Wall_0bOlnkW59DjubTltZzGQyL20220912151918
Wall_0bOlnkW59DjubTltZzGQus20220912151918
Wall_0bOlnkW59DjubTltZzGP0i20220912151918
Clutter_0bOlnkW59DjubTltZzGPIs20220912151918
Wall_0bOlnkW59DjubTltZzGPUp20220912151918
Wall_3AUUEyCM9AXg8xdG0nwgQP20220912151918
Clutter_3AUUEyCM9AXg8xdG0nwgSm20220912151918
Wall_3AUUEyCM9AXg8xdG0nwgHW20220912151918
Clutter_3AUUEyCM9AXg8xdG0nwgMm20220912151918
Clutter_3AUUEyCM9AXg8xdG0nwgMC20220912151918
Clutter_2I1MjW3CrCRgIwhVTYKlmy20220912151918
Clutter_157UZAQ917$vE3AxB$q0GF20220912151918
Wall_157UZAQ917$vE3AxB$q0Th20220912151918
Wall_0dJmiHv_TE5eZ4Ssa1hZAh20220912151918
Wall_3r0qdwUzP0HBXErMls12vf20220912151918
Wall_1r8UZLUsT3OvNNv5NT4wF920220912151918
Clutter_3dTGhr_b164vU83Wm1iBTN20220912151918
Clutter_3dTGhr_b164vU83Wm1iBUA20220912151918
Clutter_3dTGhr_b164vU83Wm1iBZQ2022091215191

In [ ]:
# succes = 0
# for n in sn.bimNodes:
#     try:
#         # print(n.overviewImagePath)
#         succes +=1
#     except: 
#         pass
#         # print("FAILED)")
# print(succes)

## Processing of the pointcloud data

1. Crop the BIM elements from a point cloud using their expanded BoundingBoxes

In [ ]:
# def create_croppedpcd(pointcloud: PointCloudNode, element: BIMNode, bb: o3d.geometry.OrientedBoundingBox, sn : SessionNode) -> PointCloudNode:
#     """Crops a part of a point cloud around a BIM object.

#     Args:
#         pointcloud (PointCloudNode): Pointcloud from which the zone needs to be cropped
#         element (BIMNode): The BIMNode that the cropped region represents
#         bb (o3d.geometry.OrientedBoundingBox): The boundingbox that needs to be cropped from the pointcloud
#         samplesize (Optional float): When the results needs to be downsampled.
            
#     Returns:
#         PointCloudNode containing the pointcloud of the cropped region
#     """
#     myCroppedPCD=pointcloud.resource.crop(bb)
#     if len(np.asarray(myCroppedPCD.points)) > 0:
#         if sn.resolution and len(np.asarray(myCroppedPCD.points)) >= 1000:
#             myCroppedPCD = myCroppedPCD.voxel_down_sample(sn.resolution)
        
#         try:
#             myCroppedPCDNode=PointCloudNode(resource=myCroppedPCD,
#                                         name = element.get_name().split("-")[0] + "-CROPPEDPCD",
#                                         subject=element.get_name().split("-")[0] + "-CROPPEDPCD",
#                                         isDerivedFromGeometry=pointcloud.subject,
#                                         processed = False)
#             myCroppedPCDNode.save_resource(sn.projectProcessingPath)
#             return myCroppedPCDNode
        
#         except:
#             return None
#     else:
#         return None

# def add_to_croppedpcd(pointcloud: PointCloudNode, element: BIMNode, target: PointCloudNode ,bb: o3d.geometry.OrientedBoundingBox, sn: SessionNode) -> PointCloudNode:
#     """adds a part to a point cloud.

#     Args:
        
            
#     Returns:
#         PointCloudNode containing the pointcloud of the cropped region
#     """
#     myCroppedPCD=pointcloud.resource.crop(bb)
#     if len(np.asarray(myCroppedPCD.points)) > 0:
#         if sn.resolution and len(np.asarray(myCroppedPCD.points)) >= 1000:
#             myCroppedPCD = myCroppedPCD.voxel_down_sample(sn.resolution)
#         target.resource.__iadd__(myCroppedPCD)
#         target.save_resource(sn.projectProcessingPath)

In [ ]:
# def get_BIMNode(subject, sn : SessionNode):
#     """Gets the BIMNode  from which it is derived.

#     Args:
#         subject: the subject from which it was derived
#         BIMNodeList (List[BIMNodes]): list of all the BIMNodes in the project
            
#     Returns:
#         PointCloudNode containing the pointcloud of the cropped region
#     """
#     for n in sn.bimNodes:
#         if n.subject is subject:
#             return n

In [ ]:
# def get_cropped_pcdNode(subject, sn : SessionNode):
#     for n in sn.meshpcdNodes:
#         if n.subject.split("-")[0] in subject:
#             return n

In [ ]:
# #reset processed
# for meshpcdNode in sn.meshpcdNodes:
#     meshpcdNode.processed = False

In [ ]:
sn.croppedpcdNodes  = []

for realPCD in sn.pcdPath:
    pointcloudnode = geomapi.nodes.PointCloudNode(path = realPCD, getResource = True)
    realorientedBB = gmu.get_oriented_bounding_box(pointcloudnode.cartesianBounds)
    realexpandedBB = gmu.expand_box(realorientedBB, u=1, v=1, w=1)

    for meshpcdNode in sn.meshpcdNodes:
        #Get the oriented boundingbox
        orientedBB = o3d.geometry.OrientedBoundingBox.get_oriented_bounding_box(meshpcdNode.resource)#gmu.get_oriented_bounding_box(meshpcdNode.cartesianBounds)
        expandedBB = gmu.expand_box(orientedBB, u=0.2, v=0.2, w=0)
        
        inliers = len(realexpandedBB.get_point_indices_within_bounding_box(expandedBB.get_box_points()))
        BIMNode = val.get_BIMNode(meshpcdNode.isDerivedFromGeometry, sn = sn)
         
        if inliers > 0 and meshpcdNode.resource is not None:
            if meshpcdNode.processed:
                
                myCroppedPCDNode = val.get_cropped_pcdNode(meshpcdNode.subject, sn = sn)
                val.add_to_croppedpcd(pointcloud = pointcloudnode, element= BIMNode, target=myCroppedPCDNode ,bb=expandedBB, sn = sn)
            else:
                myCroppedPCDNode = val.create_croppedpcd(pointcloud = pointcloudnode, element = BIMNode,  bb=expandedBB, sn = sn)
                if myCroppedPCDNode is not None:
                    sn.croppedpcdNodes.append(myCroppedPCDNode)
                    meshpcdNode.processed = True
        

In [ ]:
# print(len(sn.croppedpcdNodes)) #232 bestanden
# print(len(sn.meshpcdNodes)) #241 bestanden

In [ ]:
# #Check which elements where not cropped from the pointclouds
# for meshpcdnode in meshpcdNodes:
#     if not meshpcdnode.processed:
#         print(meshpcdnode.name)
#         # o3d.visualization.draw_geometries([meshpcdnode.resource, pointcloudnode.resource])
#         meshpcdnode.save_resource(projectPath)


2. Filter the Cropped Pointcloud to remove as much Clutter as possible

In [ ]:
def get_referenceCloudNode(subject, sn : SessionNode):
    succ = False
    for n in sn.meshpcdNodes:
        if n.subject.split("-")[0] in subject:
            succ = True
            return n
    if not succ:
        print(subject)


2.1 Filtering on distance

In [ ]:
# def distance_filtering(target: PointCloudNode, reference: PointCloudNode, sn : SessionNode):
    
#     d = target.resource.compute_point_cloud_distance(reference.resource)
    
#     i = 0
#     indeces = []

#     while i < len(d):
#         if d[i] < sn.Td:
#             indeces.append(i)
#         i += 1
    
#     BIMNode = get_BIMNode(reference.isDerivedFromGeometry, sn=sn)
    
#     if len(indeces) > 50:
#         filteredpcdNode = PointCloudNode(resource = target.resource.select_by_index(indeces),
#                                         subject=BIMNode.get_name().split("-")[0] + "-FILTEREDPCD",
#                                         name = BIMNode.get_name().split("-")[0] + "-FILTEREDPCD",
#                                         isDerivedFromGeometry=target.subject,
#                                         processingType = 'DISTANCE',
#                                         processed = False)
#         filteredpcdNode.path = os.path.join(sn.projectProcessingPath, filteredpcdNode.get_name() + ".pcd")
#         if os.path.exists(filteredpcdNode.path):
#             print("Path already exisits")
#             filteredpcdNode.path = os.path.join(sn.projectProcessingPath, filteredpcdNode.get_name() + time.strftime("%Y%m%d%H%M%S") +".pcd")
#         filteredpcdNode.save_resource(sn.projectProcessingPath)
       
#         return filteredpcdNode
#     else:
#         return None

2.2 Filtering on distance and normal


In [ ]:
# def normal_filtering(target: PointCloudNode, reference: PointCloudNode, sn : SessionNode):
#     if not target.resource.has_normals():
#         target.resource.estimate_normals()
#     if not reference.resource.has_normals():
#         reference.resource.estimate_normals()

#     d = target.resource.compute_point_cloud_distance(reference.resource)

#     i = 0
#     indeces_d = []

#     while i < len(d):
#         if d[i] < sn.Td:
#             indeces_d.append(i)
#         i += 1
    
#     BIMNode = get_BIMNode(reference.isDerivedFromGeometry, sn=sn)
    
#     if len(indeces_d) > 50:
#         indeces_n = []
#         kdtree = o3d.geometry.KDTreeFlann(reference.resource)
#         for id in indeces_d:
#             [k, idx, d] = kdtree.search_radius_vector_3d(target.resource.points[id], 0.1)
#             matched = False
#             i = 0
#             while not matched and i < len(idx) and len(idx) > 0:
#                 if np.abs(np.dot(np.asarray(target.resource.normals[id]), np.asarray(reference.resource.normals[idx[i]]))) > sn.Tdot:
#                     indeces_n.append(id)
#                     matched = True
#                 i += 1
#         if len(indeces_n) > 50:
#             filteredpcdNode = PointCloudNode(resource = target.resource.select_by_index(indeces_n),
#                                         subject=BIMNode.get_name().split("-")[0] + "-FILTEREDPCD",
#                                         name = BIMNode.get_name().split("-")[0] + "-FILTEREDPCD",
#                                         isDerivedFromGeometry=target.subject,
#                                         processingType = 'NORMALS',
#                                         processed = False)
#             filteredpcdNode.path = os.path.join(sn.projectProcessingPath, filteredpcdNode.get_name() + ".pcd")
#             if os.path.exists(filteredpcdNode.path):
#                 print("Path already exisits")
#                 filteredpcdNode.path = os.path.join(sn.projectProcessingPath, filteredpcdNode.get_name() + time.strftime("%Y%m%d%H%M%S") +".pcd")
#             filteredpcdNode.save_resource(sn.projectProcessingPath)
        
#             return filteredpcdNode
#         else: 
#             return None
#     else:
#         return None

In [ ]:
sn.filteredPCDList1 = []

for pcdNode in sn.croppedpcdNodes:
    referenceNode = get_referenceCloudNode(subject = pcdNode.subject, sn= sn)
    try:
        # print(referenceNode.name)
        # filteredPCD = distance_filtering(target = pcdNode, reference = referenceNode, Treshold= 1)
        filteredPCD = val.normal_filtering(target =pcdNode, reference = referenceNode, sn = sn)
        if filteredPCD is not None:
            sn.filteredPCDList1.append(filteredPCD)
    except:
        print(pcdNode.subject)

# print(len(sn.filteredPCDList1))

## Quality Control
1. Compute the LOA per element

In [ ]:
# def color_by_LOA(target: PointCloudNode, distances: o3d.utility.DoubleVector, sn : SessionNode):
#     i=0
#     target.resource.paint_uniform_color([1,1,1])
#     # print(len(np.asarray(target.resource.points)))
#     # print(len(np.asarray(target.resource.colors)))
#     # print(len(np.asarray(distances)))
#     while i < len(distances):
#         distance = distances[i]
        
#         if distance < sn.t00:
#             np.asarray(target.resource.colors)[i] = [1,0,0]
#         if distance < sn.t10:
#             np.asarray(target.resource.colors)[i] = [1,0.76,0]
#         if distance < sn.t20:
#             np.asarray(target.resource.colors)[i] = [1,1,0]
#         if distance < sn.t30:
#             np.asarray(target.resource.colors)[i] = [0,1,0]
#         i += 1
    
#     pcdResultDirectory = os.path.join(sn.resultsPath, "PCD")
#     if not os.path.isdir(pcdResultDirectory): #check if the folder exists
#         os.mkdir(pcdResultDirectory)

#     target.save_resource(pcdResultDirectory)
        

In [ ]:
# def compute_LOA(target: PointCloudNode, reference: PointCloudNode,sn : SessionNode):
#     d = target.resource.compute_point_cloud_distance(reference.resource)
    
#     LOA10Inliers = 0
#     LOA20Inliers = 0
#     LOA30Inliers = 0
#     LOA00Inliers = 0
#     totalPoints = len(d)

#     for dis in np.asarray(d):
#         if dis < sn.t00:
#             LOA00Inliers += 1
#         if dis < sn.t10:
#             LOA10Inliers += 1
#         if dis < sn.t20:
#             LOA20Inliers += 1
#         if dis < sn.t30: 
#             LOA30Inliers += 1
#     if LOA00Inliers < totalPoints:
#         prct = LOA00Inliers/totalPoints *100
    
#     if LOA00Inliers > 0:
#         LOA10 = LOA10Inliers/LOA00Inliers
#         LOA20 = LOA20Inliers/LOA00Inliers
#         LOA30 = LOA30Inliers/LOA00Inliers

#         color_by_LOA(target = target, distances=d, sn = sn)

#         return [LOA10, LOA20, LOA30]
#     else:
#         return None
        

In [ ]:
# def LOA_to_csv(element : BIMNode, sn : SessionNode):
#     data = [element.name, element.globalId, element.key, element.accuracy, element.LOA10, element.LOA20, element.LOA30]
#     sn.csvWriter.writerow(data)

# def LOA_to_xlsx(element : BIMNode, sn : SessionNode):

#     sn.worksheet.write(sn.xlsxRow, 0, element.name)
#     sn.worksheet.write(sn.xlsxRow, 1, element.globalId)
#     sn.worksheet.write(sn.xlsxRow, 2, element.key)
#     sn.worksheet.write(sn.xlsxRow, 3, element.accuracy)
#     sn.worksheet.write(sn.xlsxRow, 4, element.LOA10)
#     sn.worksheet.write(sn.xlsxRow, 5, element.LOA20)
#     sn.worksheet.write(sn.xlsxRow, 6, element.LOA30)

#     sn.xlsxRow +=1


# def LOA_to_mesh(element : BIMNode, sn : SessionNode):
    
#     meshResultDirectory = os.path.join(sn.resultsPath, "PLY")
#     if not os.path.isdir(meshResultDirectory): #check if the folder exists
#         os.mkdir(meshResultDirectory)
    
#     meshResultName = element.name
#     meshResultFileName = meshResultName + ".ply"
#     meshResultPath = os.path.join(meshResultDirectory, meshResultFileName)

#     if not element.accuracy == "NO LOA":
#         if element.resource or os.path.exists(element.path):
#             loaded = False
#             if not element.resource and element.path:
#                 #Load the meshobject from disk
#                 element.resource = o3d.io.read_triangle_mesh(element.path)
#                 loaded = True
#         result = element.resource
#         if element.accuracy == "LOA30":
#             result.paint_uniform_color([0,1,0])
#         elif element.accuracy == "LOA20":
#             result.paint_uniform_color([1,1,0])
#         elif element.accuracy == "LOA10":
#             result.paint_uniform_color([1,0.76,0])
#         elif element.accuracy == "LOA00":
#             result.paint_uniform_color([1,0,0])
#         else:
#             result.paint_uniform_color([1,1,1])
        
#         if loaded:
#             element.resource = None
        
#         o3d.io.write_triangle_mesh(meshResultPath,result)
#         # print("OBJ file saved in %s" %meshResultPath)

#     else: 
#         print("ERROR no LOAs where computed in previous steps") 

In [ ]:
# def optimize_camera(node: Node):
    
#     fov = np.pi / 3 #60 #degrees
    
#     #determine extrinsic camera parameters
#     extrinsic = np.empty((1,3), dtype=float)
    
#     if node.resource is not None:
#         box=node.resource.get_oriented_bounding_box()
#         c = box.get_center()
#         u= box.extent[0]

#         d_w=math.cos(fov/2)*u
#         #determine c_i
#         rotation_matrix=box.R
#         pcd = o3d.geometry.PointCloud()
#         array=np.array([[c[0],c[1],c[2]+d_w]])
#         pcd.points = o3d.utility.Vector3dVector(array)
#         pcd.rotate(rotation_matrix, center =c)
#         c_i=np.asarray(pcd.points[0])

#         up = [0, 0, 1]  # camera orientation

#     return c,c_i,up

In [ ]:
# def create_detail_image(resultpcd : PointCloudNode, sn : SessionNode, width = 640, height = 480):

#     referenceNode = get_referenceCloudNode(subject = resultpcd.subject, sn = sn)
#     bimnode = get_BIMNode(subject = referenceNode.isDerivedFromGeometry, sn = sn)
    
#     #generate scene
#     render = o3d.visualization.rendering.OffscreenRenderer(width,height)

#     #Determine the materials for visualization 
#     mtl=o3d.visualization.rendering.MaterialRecord()
#     mtl.base_color = [1.0, 1.0, 1.0, 1.0]  # RGBA
#     mtl.shader = "defaultUnlit"
    
#     mtlline=o3d.visualization.rendering.MaterialRecord()
#     mtlline.base_color = [0.0, 0.0, 0.0, 0.0]  # RGBA
#     mtlline.shader = "defaultUnlit"
    
#     #set camera
#     # # Look at the origin from the front (along the -Z direction, into the screen), with Y as Up.
#     center, eye, up = optimize_camera(bimnode)
#     render.scene.camera.look_at(center, eye, up)
    
#     #add geometries
#     if not bimnode.resource:
#         bimnode.get_resource()
#     #Add the BIM elements wireframe to the scene to have a bether understanding of the object
#     wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(bimnode.resource)
#     wireframe.paint_uniform_color([0,0,0])
    
#     render.scene.add_geometry("test",resultpcd.resource,mtl)
#     render.scene.add_geometry("lines", wireframe, mtlline)

#     #render the image
#     img = render.render_to_image()
#     imageResultDirectory = os.path.join(sn.resultsPath, "IMAGES")
#     if not os.path.isdir(imageResultDirectory):
#         os.mkdir(imageResultDirectory)
#     detailImageResultName = bimnode.name
#     detailImageResultFileName = detailImageResultName + ".png"
#     detailImageResultPath = os.path.join(imageResultDirectory, detailImageResultFileName)

#     o3d.io.write_image(detailImageResultPath, img)
#     bimnode.detailImagePath = detailImageResultPath


In [ ]:
# def create_report(resultpcd : PointCloudNode, sn : SessionNode):

#     referenceNode = get_referenceCloudNode(subject = resultpcd.subject, sn = sn)
#     bimnode = get_BIMNode(subject = referenceNode.isDerivedFromGeometry, sn = sn)

#     #Load both the detail and overview image of the object
#     detail = Image.open(bimnode.detailImagePath)
#     overview = Image.open(bimnode.overviewImagePath)

#     info = Image.new('RGB', (int(overview.width/2), int(overview.height/2)), color = (255,255,255))
#     d = ImageDraw.Draw(info)
#     d.text((10,10),bimnode.name, fill=(0,0,0))
#     d.text((10,20), time.strftime("%Y%m%d-%H%M%S"), fill=(0,0,0))
    
#     d.text((10,70),"LOA10  (%sm - %sm)" %(sn.t20,sn.t10), fill=(0,0,0))
#     if np.round(bimnode.LOA10, decimals = 2) < sn.p10:
#         d.text((200,70), str(np.round(bimnode.LOA10*100, decimals = 2)) + "%", fill=(255,0,0))
#     else:
#         d.text((200,70), str(np.round(bimnode.LOA10*100, decimals = 2))+ "%", fill=(0,255,0))
    
#     d.text((10,80), "LOA20  (%sm - %sm)" %(sn.t30, sn.t20), fill=(0,0,0))
#     if np.round(bimnode.LOA20, decimals = 2) < sn.p20:
#         d.text((200,80), str(np.round(bimnode.LOA20*100, decimals = 2))+ "%", fill=(255,0,0))
#     else:
#         d.text((200,80), str(np.round(bimnode.LOA20*100, decimals = 2))+ "%", fill=(0,255,0))

#     d.text((10,90), "LOA30  (%sm - %sm)" %(0, sn.t30), fill=(0,0,0))
#     if np.round(bimnode.LOA30, decimals = 2) < sn.p30:
#         d.text((200,90), str(np.round(bimnode.LOA30*100, decimals = 2))+ "%", fill=(255,0,0))
#     else:
#         d.text((200,90), str(np.round(bimnode.LOA30*100, decimals = 2))+ "%", fill=(0,255,0))
    

#     resizedInfo= info.resize((overview.width,overview.height), Image.ANTIALIAS)
#     info_overview = Image.new('RGB', (overview.width, overview.height*2))
#     info_overview.paste(resizedInfo, (0,0))
#     info_overview.paste(overview, (0,overview.height))

#     newHeigth = 2*overview.height
#     hpercent = (newHeigth/float(detail.height))
#     wsize = int((float(detail.width)*float(hpercent)))
#     resizedImage = detail.resize((wsize, newHeigth), Image.ANTIALIAS)

#     report = Image.new('RGB', (overview.width + wsize, overview.height*2))
#     report.paste(info_overview,(0,0))
#     report.paste(resizedImage, (info_overview.width, 0))

#     reportsResultDirectory = os.path.join(sn.resultsPath, "Reports")
#     if not os.path.isdir(reportsResultDirectory):
#         os.mkdir(reportsResultDirectory)
#     reportName = bimnode.name
#     reportFileName = reportName + ".png"
#     reportPath = os.path.join(reportsResultDirectory, reportFileName)

#     report.save(reportPath)

In [ ]:
print(len(sn.filteredPCDList1))
for pcdNode in sn.filteredPCDList1:
    print(pcdNode)
    print(pcdNode.name)
    referenceNode = val.get_referenceCloudNode(subject = pcdNode.subject, sn = sn)
    LOAs = val.compute_LOA(target = pcdNode, reference=referenceNode, sn = sn)
    element = val.get_BIMNode(subject = referenceNode.isDerivedFromGeometry, sn = sn)
    print(element.name)
    #TODO: Consider overlap between pointclouds, how much of the object is observed to determine the reliability of the LOA prediction
    if not LOAs is None:
        element.LOA10 = LOAs[0]
        element.LOA20 = LOAs[1]
        element.LOA30 = LOAs[2]
        if LOAs[2] > sn.p30:
            element.accuracy = "LOA30"
        elif LOAs[1] > sn.p20:
            element.accuracy = "LOA20"
        elif LOAs[0] > sn.p10:
            element.accuracy = "LOA10"
        else:
            element.accuracy = "NO LOA"
    else:
        element.accuracy = "NO LOA"
        element.LOA10 = "NO"
        element.LOA20 = "NO"
        element.LOA30 = "NO"

    val.LOA_to_csv(element, sn = sn)
    val.LOA_to_xlsx(element,sn = sn)
    val.LOA_to_mesh(element, sn = sn)
    val.create_detail_image(resultpcd = pcdNode, sn = sn)
    try:
        val.create_report(pcdNode, sn=sn)
    except:
        print(pcdNode)
        print(element.name)
        print(element.resource)

csvFile.close()
sn.workbook.close()